## Importing pyspark packages

In [73]:
import numpy as np
import pandas as pd
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
 


ImportError: No module named spark.implicits._

# Uploading/ Pre-processing

### Uploading Data

In [2]:
df =spark.read.csv("/Users/arjit/Desktop/train.csv", header =True, inferSchema = True)
df.printSchema()

root
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: 

In [3]:
df.count()

37670293

##### Number of rows - 37670293 / Column -24

### Coulmns -

In [4]:
df.columns

['date_time',
 'site_name',
 'posa_continent',
 'user_location_country',
 'user_location_region',
 'user_location_city',
 'orig_destination_distance',
 'user_id',
 'is_mobile',
 'is_package',
 'channel',
 'srch_ci',
 'srch_co',
 'srch_adults_cnt',
 'srch_children_cnt',
 'srch_rm_cnt',
 'srch_destination_id',
 'srch_destination_type_id',
 'is_booking',
 'cnt',
 'hotel_continent',
 'hotel_country',
 'hotel_market',
 'hotel_cluster']

### Displaying row # 1

In [7]:
df.take(1)

[Row(user_location_country=66, user_location_region=348, user_location_city=48862, orig_destination_distance=2234.2641, user_id=12, srch_ci=datetime.datetime(2014, 8, 27, 0, 0), srch_co=datetime.datetime(2014, 8, 31, 0, 0), srch_adults_cnt=2, srch_children_cnt=0, srch_rm_cnt=1, srch_destination_id=8250, srch_destination_type_id=1, is_booking=0, cnt=3, hotel_continent=2, hotel_country=50, hotel_market=628, hotel_cluster=1)]

### Droping Irrelevant columns 

In [8]:
df = df.drop('is_mobile','site_name','posa_continent', 'is_package', 'channel','date_time')
df.printSchema()
display(df)  

root
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: integer (nullable = true)
 |-- hotel_market: integer (nullable = true)
 |-- hotel_cluster: integer (nullable = true)



DataFrame[user_location_country: int, user_location_region: int, user_location_city: int, orig_destination_distance: double, user_id: int, srch_ci: timestamp, srch_co: timestamp, srch_adults_cnt: int, srch_children_cnt: int, srch_rm_cnt: int, srch_destination_id: int, srch_destination_type_id: int, is_booking: int, cnt: int, hotel_continent: int, hotel_country: int, hotel_market: int, hotel_cluster: int]

#### Rearranging Data

In [9]:
df.count()

37670293

In [10]:
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"
timeDiff = (F.unix_timestamp(df.srch_co, format=timeFmt)
            - F.unix_timestamp(df.srch_ci, format=timeFmt))
df = df.withColumn("Num_OfDays", timeDiff/86400)

In [11]:
df.columns

['user_location_country',
 'user_location_region',
 'user_location_city',
 'orig_destination_distance',
 'user_id',
 'srch_ci',
 'srch_co',
 'srch_adults_cnt',
 'srch_children_cnt',
 'srch_rm_cnt',
 'srch_destination_id',
 'srch_destination_type_id',
 'is_booking',
 'cnt',
 'hotel_continent',
 'hotel_country',
 'hotel_market',
 'hotel_cluster',
 'Num_OfDays']

#### We can create a new dataset with variables relevant to our model

In [79]:
newDF = df.groupBy( 'srch_destination_id','is_booking','hotel_cluster').count().orderBy('srch_destination_id')
newDF.take(10)

[Row(srch_destination_id=0, is_booking=0, hotel_cluster=3, count=2),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=57, count=1),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=20, count=4),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=30, count=2),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=20, count=22),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=30, count=20),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=60, count=17),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=5, count=1),
 Row(srch_destination_id=2, is_booking=1, hotel_cluster=20, count=1),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=20, count=3)]

In [70]:
newDF.printSchema

<bound method DataFrame.printSchema of DataFrame[srch_destination_id: int, is_booking: int, hotel_cluster: int, count: bigint, weight: double]>

In [81]:
newdf2.take(5)

[Row(srch_destination_id=148, collect_set(hotel_cluster)=[15, 30, 9, 60, 51, 16, 32, 39, 76, 91, 40, 47, 48, 35, 50, 6, 57, 72, 36, 42, 28, 7, 94, 14, 8, 59, 23]),
 Row(srch_destination_id=463, collect_set(hotel_cluster)=[30, 9, 78, 3, 36, 62, 44]),
 Row(srch_destination_id=471, collect_set(hotel_cluster)=[33, 9, 78, 99, 82, 97, 22, 58, 14, 29]),
 Row(srch_destination_id=496, collect_set(hotel_cluster)=[51, 15, 60, 16, 53, 32, 83, 68, 18, 10, 91, 47, 77, 48, 13, 42, 93, 72, 28, 7, 94, 14, 23]),
 Row(srch_destination_id=833, collect_set(hotel_cluster)=[35, 53, 89, 3])]

### Putting weights on booking and clicks

In [18]:
newDF = newDF.orderBy(newDF['srch_destination_id'],newDF['count'].desc())
newDF.take(10)

[Row(srch_destination_id=0, is_booking=0, hotel_cluster=3, count=2),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=20, count=22),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=30, count=20),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=60, count=17),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=20, count=4),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=30, count=2),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=57, count=1),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=30, count=5),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=20, count=3),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=5, count=1)]

In [89]:
newDF = newDF.withColumn('weight',1.2*newDF['is_booking']+0.06248*newDF['count']+0.0005*newDF['hotel_cluster'])
newDF.take(10)

[Row(srch_destination_id=0, is_booking=0, hotel_cluster=3, count=2, weight=0.12646),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=30, count=2, weight=1.3399599999999998),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=60, count=17, weight=1.09216),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=20, count=4, weight=1.4599199999999999),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=20, count=22, weight=1.38456),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=57, count=1, weight=0.09098),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=30, count=20, weight=1.2646),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=46, count=1, weight=0.08548),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=5, count=1, weight=0.06498),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=20, count=3, weight=0.19744)]

In [24]:
newDF.count()

578356

In [18]:
value = newDF.count()

578356

In [90]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(newDF['srch_destination_id']).orderBy(newDF['srch_destination_id'],newDF['weight'].desc())

newDF1 = newDF.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 5)

In [91]:
newDF1 = newDF1.orderBy(newDF1['srch_destination_id'],newDF1['weight'].desc())
newDF2 = newDF1
newDF1.take(20)

[Row(srch_destination_id=0, is_booking=0, hotel_cluster=3, count=2, weight=0.12646, rank=1),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=20, count=4, weight=1.4599199999999999, rank=1),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=20, count=22, weight=1.38456, rank=2),
 Row(srch_destination_id=1, is_booking=1, hotel_cluster=30, count=2, weight=1.3399599999999998, rank=3),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=30, count=20, weight=1.2646, rank=4),
 Row(srch_destination_id=1, is_booking=0, hotel_cluster=60, count=17, weight=1.09216, rank=5),
 Row(srch_destination_id=2, is_booking=1, hotel_cluster=20, count=1, weight=1.27248, rank=1),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=30, count=5, weight=0.3274, rank=2),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=20, count=3, weight=0.19744, rank=3),
 Row(srch_destination_id=2, is_booking=0, hotel_cluster=53, count=1, weight=0.08898, rank=4),
 Row(srch_destination_id=2, is_booking

In [92]:
newdf2 = newDF1.groupBy('srch_destination_id').agg(collect_set('hotel_cluster'))

In [93]:
newdf2 = newdf2.orderBy(newdf2['srch_destination_id'])

In [94]:
newdf2.take(5)

[Row(srch_destination_id=0, collect_set(hotel_cluster)=[3]),
 Row(srch_destination_id=1, collect_set(hotel_cluster)=[30, 60, 20]),
 Row(srch_destination_id=2, collect_set(hotel_cluster)=[30, 46, 20, 53]),
 Row(srch_destination_id=3, collect_set(hotel_cluster)=[60, 53]),
 Row(srch_destination_id=4, collect_set(hotel_cluster)=[78, 67, 82, 32, 25])]

In [95]:
newdf2.printSchema

<bound method DataFrame.printSchema of DataFrame[srch_destination_id: int, collect_set(hotel_cluster): array<int>]>

In [96]:
newdf2.toPandas()

,srch_destination_id,collect_set(hotel_cluster)
0,0,[3]
1,1,"[30, 60, 20]"
2,2,"[30, 46, 20, 53]"
3,3,"[60, 53]"
4,4,"[78, 67, 82, 32, 25]"
5,5,[89]
6,6,[89]
7,7,"[16, 42, 94]"
8,8,"[48, 42, 7, 43, 91]"
9,9,"[20, 75, 61, 40]"


In [98]:
test_df =spark.read.csv("/Users/arjit/Desktop/test.csv", header =True, inferSchema = True)

In [99]:
test_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: integer (nullable = true)
 |-- hotel_market: in

In [100]:
test_df.take(5)

[Row(id=0, date_time=datetime.datetime(2015, 9, 3, 17, 9, 54), site_name=2, posa_continent=3, user_location_country=66, user_location_region=174, user_location_city=37449, orig_destination_distance=5539.0567, user_id=1, is_mobile=1, is_package=0, channel=3, srch_ci=u'2016-05-19', srch_co=datetime.datetime(2016, 5, 23, 0, 0), srch_adults_cnt=2, srch_children_cnt=0, srch_rm_cnt=1, srch_destination_id=12243, srch_destination_type_id=6, hotel_continent=6, hotel_country=204, hotel_market=27),
 Row(id=1, date_time=datetime.datetime(2015, 9, 24, 17, 38, 35), site_name=2, posa_continent=3, user_location_country=66, user_location_region=174, user_location_city=37449, orig_destination_distance=5873.2923, user_id=1, is_mobile=1, is_package=0, channel=10, srch_ci=u'2016-05-12', srch_co=datetime.datetime(2016, 5, 15, 0, 0), srch_adults_cnt=2, srch_children_cnt=0, srch_rm_cnt=1, srch_destination_id=14474, srch_destination_type_id=7, hotel_continent=6, hotel_country=204, hotel_market=1540),
 Row(id=2